# CatBoost

In [ ]:
!pip install catboost scikit-learn

In [1]:
import catboost as cat
from catboost import CatBoostClassifier
from catboost import Pool

import pickle
import gc

ModuleNotFoundError: No module named 'catboost'

In [ ]:
#カテゴリー変数を格納
cat_features = list(range(0, x.shape[1]))


#訓練データと評価用データを分割
x_train, x_vali, y_train, y_vali = train_test_split(x, y,test_size=0.3)


#バリデーションを行うための用意
train_pool = Pool(x_train, y_train)
validate_pool = Pool(x_vali, y_vali)


#モデルの作成
ctb= cat.CatBoostClassifier(eval_metric="AUC", depth=7, iterations= 200, 
                            l2_leaf_reg= 1, learning_rate= 0.05, logging_level='Silent')


#モデルにより学習を行う
ctb_model = ctb.fit(x_train,y_train, cat_features= cat_features,eval_set=(x_vali, y_vali))

print("AUC =", auc(ctb_model, x_train, x_vali))

In [ ]:
#モデルによる予測
y_pred = ctb_model.predict_proba(x_test1)

# CatBoostの関数

In [ ]:
cat_params = {
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'num_boost_round': 10000,
        'learning_rate': 0.01,
        'random_state': 42,
        'task_type': 'CPU',
        'depth': 6,
    }

In [ ]:
def catboost(X, y, params, folds, categorycal_list=[], add_suffix=''):
    
    '''
    cat_params = {
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'num_boost_round': 10000,
        'learning_rate': 0.03,
        'random_state': 42,
        'task_type': 'CPU',
        'depth': 6,
    }
    '''
    
    oof_pred = np.zeros(len(y), dtype=np.float32)

    fold_unique = sorted(folds.unique())
    for fold in fold_unique:
        idx_train = (folds!=fold)
        idx_valid = (folds==fold)
        
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        cat_train = cat.Pool(
            x_train, 
            label=y_train,
            cat_features=categorycal_list,
        )
        
        cat_valid = cat.Pool(
            x_valid, 
            label=y_valid,
            cat_features=categorycal_list,
        )
        
        #modelの定義
        model = cat.CatBoostClassifier(**params)
        
        #modelの学習
        model.fit(
            cat_train,
            early_stopping_rounds=100,
            plot=False,
            use_best_model=True,
            eval_set=[cat_valid],
            verbose=100
        )
        
        #pickle.dumpにより，指定したファイルにオブジェクトを保存(pickle化)
        #open関数でファイルを作成，または上書き保存
        pickle.dump(model, open(f'cat_fold{fold}{add_suffix}.pkl', 'wb'))
        
        #予測値の出力
        pred_i = model.predict_proba(cat_valid)[:, 1]
        
        oof_pred[x_valid.index] = pred_i
        
        #round関数で小数点の四捨五入
        #AUCの計算
        score = round(roc_auc_score(y_valid, pred_i), 5)
        print(f'Performance of the prediction: {score}\n')

    score = round(roc_auc_score(y, oof_pred), 5)
    print(f'All Performance of the prediction: {score}')
    #モデルの削除
    del model
    #ガベージコレクション(pythonには備わっているので使わなくても良い?)
    gc.collect()
    return oof_pred

# CatBoostの予測の関数

In [ ]:
def pred_catboost(X, data_dir: Path, add_suffix=''):
    #pickleファイルからモデルを読み込む
    models = glob(str(data_dir / f'cat*{add_suffix}.pkl'))
    #非pickle化
    models = [pickle.load(open(model, 'rb')) for model in models]
    #各モデルでの出力値をリストに格納
    preds = np.array([model.predict_proba(X)[:, 1] for model in models])
    #平均を取る
    preds = np.mean(preds, axis=0)
    #予測値の出力
    return preds